In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#!echo "Listing dataset path:"
#!ls -R "/content/drive/MyDrive/pascal_dataset" | head -n 40


In [ ]:
'''https://www.kaggle.com/datasets/sakshamjn/vehicle-detection-8-classes-object-detection'''
#import kagglehub
#path = kagglehub.dataset_download("sakshamjn/vehicle-detection-8-classes-object-detection")
#print("Path to dataset files:", path)

'https://www.kaggle.com/datasets/sakshamjn/vehicle-detection-8-classes-object-detection'

In [ ]:
#!ls -R "/root/.cache/kagglehub/datasets/sakshamjn/vehicle-detection-8-classes-object-detection/versions/1" | head -n 50

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Paths
base_path = "/content/drive/MyDrive/pascal dataset/train"
img_dir = os.path.join(base_path, "images")
labels_path = os.path.join(base_path, "labels.npy")
output_base = "/content/processed_dataset"

# Class mapping from dataset (8 classes)
class_names = [
    "car",
    "light_motor_vehicle",
    "multi-axle",
    "auto",
    "truck",
    "bus",
    "motorcycle",
    "tractor"
]

# Load labels
labels_data = np.load(labels_path, allow_pickle=True)
image_files = sorted(os.listdir(img_dir))
print(f"📊 Found {len(image_files)} images and {len(labels_data)} label entries")

# Create output folders
for c in class_names:
    os.makedirs(os.path.join(output_base, c), exist_ok=True)

# Crop objects
for img_name, label_array in tqdm(zip(image_files, labels_data), total=len(image_files)):
    img_path = os.path.join(img_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue
    h, w = img.shape[:2]

    for label in label_array:
        cls_id, x_center, y_center, bbox_w, bbox_h = label
        cls_id = int(cls_id)
        cls_name = class_names[cls_id]

        x1 = int((x_center - bbox_w / 2) * w)
        y1 = int((y_center - bbox_h / 2) * h)
        x2 = int((x_center + bbox_w / 2) * w)
        y2 = int((y_center + bbox_h / 2) * h)

        # clamp to image
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w - 1, x2), min(h - 1, y2)

        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            continue

        out_path = os.path.join(output_base, cls_name, f"{os.path.splitext(img_name)[0]}_{x1}_{y1}.jpg")
        cv2.imwrite(out_path, crop)

print(f"✅ Cropping complete. Dataset ready at: {output_base}")


# --------------------------------------------------------
#                FEATURE EXTRACTION + SVM
# --------------------------------------------------------

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# Path to processed cropped dataset
dataset_path = "/content/processed_dataset"

# Only use selected classes
selected_classes = ["car", "bus", "truck"]

# Verify
available = os.listdir(dataset_path)
missing = [c for c in selected_classes if c not in available]
if missing:
    raise ValueError(f"❌ Missing classes in dataset: {missing}")

print("📁 Selected classes:", selected_classes)

# HOG descriptor
hog = cv2.HOGDescriptor()

def extract_features_and_labels(base_path, class_list):
    X, y = [], []
    for label_idx, class_name in enumerate(class_list):
        class_folder = os.path.join(base_path, class_name)
        for fname in tqdm(os.listdir(class_folder), desc=f"Extracting {class_name}"):
            fpath = os.path.join(class_folder, fname)
            img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (64, 128))
            features = hog.compute(img).flatten()
            X.append(features)
            y.append(label_idx)
    return np.array(X), np.array(y)

# Extract features
X, y = extract_features_and_labels(dataset_path, selected_classes)
print(f"📊 Total samples: {len(X)}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---------------- SVM ----------------
print("🚀 Training Linear SVM...")
clf = LinearSVC(max_iter=10000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("\n📈 Classification Report (SVM):")
print(classification_report(y_test, y_pred, target_names=selected_classes))
print("✅ SVM Accuracy:", accuracy_score(y_test, y_pred))


# --------------------------------------------------------
#             ⭐ ADDED: BAGGING ENSEMBLE ⭐
# --------------------------------------------------------

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

print("\n🚀 Training Bagging Ensemble Classifier...")

bagging_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=150,
    max_samples=0.8,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)

bagging_clf.fit(X_train, y_train)

# Evaluate Bagging
y_pred_bag = bagging_clf.predict(X_test)

print("\n📈 Classification Report (Bagging):")
print(classification_report(y_test, y_pred_bag, target_names=selected_classes))

print("✅ Bagging Accuracy:", accuracy_score(y_test, y_pred_bag))


📊 Found 1643 images and 8218 label entries


100%|██████████| 1643/1643 [01:03<00:00, 25.89it/s]


✅ Cropping complete. Dataset ready at: /content/processed_dataset
📁 Selected classes: ['car', 'bus', 'truck']


Extracting truck: 100%|██████████| 2192/2192 [00:01<00:00, 1779.92it/s]


📊 Total samples: 2611
🚀 Training Linear SVM...

📈 Classification Report (SVM):
              precision    recall  f1-score   support

         car       0.88      0.71      0.79        42
         bus       0.98      0.98      0.98        42
       truck       0.97      0.99      0.98       439

    accuracy                           0.97       523
   macro avg       0.94      0.89      0.92       523
weighted avg       0.97      0.97      0.97       523

✅ SVM Accuracy: 0.9674952198852772

🚀 Training Bagging Ensemble Classifier...

📈 Classification Report (Bagging):
              precision    recall  f1-score   support

         car       0.79      0.26      0.39        42
         bus       1.00      0.62      0.76        42
       truck       0.91      1.00      0.95       439

    accuracy                           0.91       523
   macro avg       0.90      0.63      0.70       523
weighted avg       0.91      0.91      0.89       523

✅ Bagging Accuracy: 0.9101338432122371
